In [189]:
import pickle
import difflib
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

In [190]:
!gdown https://drive.google.com/file/d/1hM2BXxXeQemLwr5UWNLvXT7GB1Zmhxac/view?usp=sharing

/usr/local/lib/python3.10/dist-packages/gdown/parse_url.py:35: UserWarning: You specified a Google Drive link that is not the correct link to download a file. You might want to try `--fuzzy` option or the following url: https://drive.google.com/uc?id=1hM2BXxXeQemLwr5UWNLvXT7GB1Zmhxac
  warnings.warn(
Downloading...
From: https://drive.google.com/file/d/1hM2BXxXeQemLwr5UWNLvXT7GB1Zmhxac/view?usp=sharing
To: /content/view?usp=sharing
84.7kB [00:00, 55.0MB/s]


In [191]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [192]:
df = pd.read_csv('/content/drive/MyDrive/MSIB/wisata_bandung.csv')
df

,category,tags,image,price,distance,rating,place_name,city,description
0,Cagar Alam,"Cagar Alam, Sedang, Gunung, Gunung Tangkuban P...",https://akcdn.detik.net.id/visual/2017/12/26/7...,30000,jauh,4.5,Gunung Tangkuban Perahu,"Cikahuripan, Kec. Lembang, Kabupaten Bandung B...",Gunung Tangkuban Parahu adalah salah satu gunu...
1,Budaya,"Budaya, Dekat dari Bandung, Budaya Gratis, Bra...",https://www.indonesia.travel/content/dam/indtr...,0,dekat,4.7,Jalan Braga,"Kec. Sumur Bandung, Kota Bandung, Jawa Barat",Jalan Braga adalah nama sebuah jalan utama di ...
2,Budaya,"Budaya, Dekat dari Bandung, Murah, Gedung Sate",https://upload.wikimedia.org/wikipedia/commons...,5000,dekat,4.6,Gedung Sate,"Jl. Diponegoro No.22, Citarum, Kec. Bandung We...","Gedung Sate, dengan ciri khasnya berupa orname..."
3,Taman Hiburan,"Taman Hiburan, Dekat dari Bandung, Mahal, TSM,...",https://www.hu-pakuan.com/media/original/20102...,280000,dekat,4.5,Trans Studio Bandung,"Jl. Gatot Subroto No.289A, Cibangkong, Kec. Ba...",Trans Studio Bandung adalah kawasan wisata ter...
4,Cagar Alam,"Cagar Alam, Bandung, Dekat, Murah, Taman Hutan...",https://assets-a1.kompasiana.com/items/album/2...,15000,dekat,4.5,Taman Hutan Raya Ir. H. Djuanda,"Kompleks Tahura, Jl. Ir. H. Juanda No.99, Cibu...",Taman Hutan Raya Ir. H. Djuanda (Tahura Djuand...
...,...,...,...,...,...,...,...,...,...
82,Cagar Alam,"Gunung Patuha, Gunung, Murah, Bandung, Alam, C...",https://pariwisataindonesia.id/wp-content/uplo...,10000,dekat,4.5,Gunung Patuha,"Ciwidey, Bandung Jawa Barat, Indonesia","Gunung Patuha terletak di kawasan Ciwidey, Kab..."
83,Cagar Alam,"Gunung Burangrang, Cagar Alam, Alam, Murah, Ba...",https://superlive.id/storage/superadventure/20...,15000,jauh,4.5,Gunung Burangrang,Barat Laut Bandung Raya,"Terletak di kawasan barat laut Bandung Raya, G..."
84,Cagar Alam,"Gunung Manglayang, Cagar Alam, Alam, Murah, Ba...",https://upload.wikimedia.org/wikipedia/commons...,10000,jauh,4.5,Gunung Manglayang,Bandung Timur,"Gunung Manglayang, yang berada di Bandung timu..."
85,Cagar Alam,"Lactasari Mini Farm dan Bird Park, Cagar Alam,...",https://www.pegipegi.com/travel/wp-content/upl...,60000,jauh,4.3,Lactasari Mini Farm dan Bird Park,"Paris Van Java, Jl. Sukajadi No.131-139, Ciped...",Lactasari Mini Farm dan Bird Park adalah tempa...


In [193]:
df_filter = df[['place_name', 'tags', 'description']]
df_filter.head()

,place_name,tags,description
0,Gunung Tangkuban Perahu,"Cagar Alam, Sedang, Gunung, Gunung Tangkuban P...",Gunung Tangkuban Parahu adalah salah satu gunu...
1,Jalan Braga,"Budaya, Dekat dari Bandung, Budaya Gratis, Bra...",Jalan Braga adalah nama sebuah jalan utama di ...
2,Gedung Sate,"Budaya, Dekat dari Bandung, Murah, Gedung Sate","Gedung Sate, dengan ciri khasnya berupa orname..."
3,Trans Studio Bandung,"Taman Hiburan, Dekat dari Bandung, Mahal, TSM,...",Trans Studio Bandung adalah kawasan wisata ter...
4,Taman Hutan Raya Ir. H. Djuanda,"Cagar Alam, Bandung, Dekat, Murah, Taman Hutan...",Taman Hutan Raya Ir. H. Djuanda (Tahura Djuand...


In [194]:
df_filter.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87 entries, 0 to 86
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   place_name   87 non-null     object
 1   tags         87 non-null     object
 2   description  87 non-null     object
dtypes: object(3)
memory usage: 2.2+ KB


In [195]:
df.shape

(87, 9)

In [196]:
df_filter = df_filter.dropna(axis='rows')
df_filter = df_filter.fillna('unknown')
df_filter

,place_name,tags,description
0,Gunung Tangkuban Perahu,"Cagar Alam, Sedang, Gunung, Gunung Tangkuban P...",Gunung Tangkuban Parahu adalah salah satu gunu...
1,Jalan Braga,"Budaya, Dekat dari Bandung, Budaya Gratis, Bra...",Jalan Braga adalah nama sebuah jalan utama di ...
2,Gedung Sate,"Budaya, Dekat dari Bandung, Murah, Gedung Sate","Gedung Sate, dengan ciri khasnya berupa orname..."
3,Trans Studio Bandung,"Taman Hiburan, Dekat dari Bandung, Mahal, TSM,...",Trans Studio Bandung adalah kawasan wisata ter...
4,Taman Hutan Raya Ir. H. Djuanda,"Cagar Alam, Bandung, Dekat, Murah, Taman Hutan...",Taman Hutan Raya Ir. H. Djuanda (Tahura Djuand...
...,...,...,...
82,Gunung Patuha,"Gunung Patuha, Gunung, Murah, Bandung, Alam, C...","Gunung Patuha terletak di kawasan Ciwidey, Kab..."
83,Gunung Burangrang,"Gunung Burangrang, Cagar Alam, Alam, Murah, Ba...","Terletak di kawasan barat laut Bandung Raya, G..."
84,Gunung Manglayang,"Gunung Manglayang, Cagar Alam, Alam, Murah, Ba...","Gunung Manglayang, yang berada di Bandung timu..."
85,Lactasari Mini Farm dan Bird Park,"Lactasari Mini Farm dan Bird Park, Cagar Alam,...",Lactasari Mini Farm dan Bird Park adalah tempa...


## Pemodelan

In [197]:
df_Tags = df.tags
df_Tags

0     Cagar Alam, Sedang, Gunung, Gunung Tangkuban P...
1     Budaya, Dekat dari Bandung, Budaya Gratis, Bra...
2        Budaya, Dekat dari Bandung, Murah, Gedung Sate
3     Taman Hiburan, Dekat dari Bandung, Mahal, TSM,...
4     Cagar Alam, Bandung, Dekat, Murah, Taman Hutan...
                            ...                        
82    Gunung Patuha, Gunung, Murah, Bandung, Alam, C...
83    Gunung Burangrang, Cagar Alam, Alam, Murah, Ba...
84    Gunung Manglayang, Cagar Alam, Alam, Murah, Ba...
85    Lactasari Mini Farm dan Bird Park, Cagar Alam,...
86    Zoomanji Bandung, Cagar Alam, Kebun Binatang, ...
Name: tags, Length: 87, dtype: object

In [198]:
def remove_brackets(data):
    data = str(data).lower()
    data = data.replace(',', '')
    data = data.replace("'", '')
    return data

df_Tags = df_Tags.apply(remove_brackets)
df_Tags

0      cagar alam sedang gunung gunung tangkuban parahu
1     budaya dekat dari bandung budaya gratis braga ...
2           budaya dekat dari bandung murah gedung sate
3     taman hiburan dekat dari bandung mahal tsm tra...
4     cagar alam bandung dekat murah taman hutan ray...
                            ...                        
82    gunung patuha gunung murah bandung alam cagar ...
83    gunung burangrang cagar alam alam murah bandun...
84    gunung manglayang cagar alam alam murah bandun...
85    lactasari mini farm dan bird park cagar alam k...
86    zoomanji bandung cagar alam kebun binatang hew...
Name: tags, Length: 87, dtype: object

In [199]:
vector_tfidf = TfidfVectorizer()
result_tfidf = vector_tfidf.fit_transform(df_Tags)

result_arr = result_tfidf.toarray()
result_arr

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.53013368]])

In [200]:
vector_tfidf.get_feature_names_out()

array(['ade', 'africa', 'afrika', 'air', 'al', 'alam', 'amazing', 'anak',
       'and', 'angklung', 'angkringan', 'art', 'artha', 'asia', 'atas',
       'baduga', 'bakery', 'balai', 'bambu', 'bandara', 'bandung', 'batu',
       'belanda', 'bermain', 'binatang', 'bintang', 'bird', 'braga',
       'budaya', 'bukit', 'bunga', 'burangrang', 'cafe', 'cagar',
       'camping', 'cepat', 'china', 'cikole', 'citatah', 'city',
       'citylight', 'ciumbuleuit', 'ciwidey', 'creative', 'curug', 'dago',
       'dan', 'dari', 'dekat', 'dewi', 'dimensi', 'djuanda', 'dreampark',
       'dunia', 'dusun', 'ecopark', 'edelweis', 'estetik', 'farm',
       'feeding', 'forest', 'fossil', 'foto', 'garden', 'gedung',
       'geologi', 'gereja', 'gkim', 'glamping', 'goa', 'gondola',
       'gratis', 'great', 'ground', 'gua', 'gunung', 'haji', 'hall',
       'harimau', 'hewan', 'hiburan', 'hobbit', 'house', 'hutan',
       'ibadah', 'im', 'interaktif', 'ir', 'irma', 'istana', 'itb',
       'jabbar', 'jalan', 'j

In [201]:
# choose one or more category
test = 'alam terdekat gratis'

In [202]:
test = 'alam sedang'
test_arr = vector_tfidf.transform([test])
test_arr = test_arr.toarray()
test_arr

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.5623888 , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.  

In [203]:
result = {}
for id, vector in enumerate(result_arr):
    cosine_val = cosine_similarity([result_arr[id]], test_arr)
    result[id] = cosine_val

result_desc = dict(sorted(result.items(), key=lambda item: item[1], reverse=True))

top_n = 5

for n, place_id in enumerate(result_desc):
    if n > 4:
        break
    print(df_filter.iloc[place_id]['place_name'])

Bukit Bintang
Curug Cipanas
Jendela Alam
Selasar Sunaryo Art Space
Gunung Tangkuban Perahu


##save data / model


In [204]:
# Save dataframe
pickle.dump(result_arr, open('resulttfidf_wisata.pkl', 'wb'))

In [205]:
resulttfidf_wisata = pickle.load(open('resulttfidf_wisata.pkl','rb'))
resulttfidf_wisata

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.53013368]])

In [206]:
pickle.dump(df, open('df.pkl', 'wb'))

In [207]:
pickle.dump(vector_tfidf, open('vector_tfidf.pkl', 'wb'))

In [208]:
vectortfidf_wisata = pickle.load(open('vector_tfidf.pkl','rb'))
vectortfidf_wisata

TfidfVectorizer()